In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8206,2021-03-20T17:00:00,ITA,19,Sicilia,38.115697,13.362357,731,122,853,14880,...,NaN,6.0,NaN,NaN,165140.0,0.0,1911228.0,1014569.0,ITG,ITG1
8207,2021-03-20T17:00:00,ITA,9,Toscana,43.769231,11.255889,1424,240,1664,24387,...,NaN,9.0,NaN,NaN,179526.0,1472.0,2754816.0,423485.0,ITI,ITI1
8208,2021-03-20T17:00:00,ITA,10,Umbria,43.106758,12.388247,372,74,446,5133,...,"Si fa presente che 7 dei ricoveri NON UTI, non...",4.0,NaN,NaN,49082.0,0.0,757297.0,168675.0,ITI,ITI2
8209,2021-03-20T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,25,2,27,476,...,NaN,0.0,NaN,NaN,8529.0,46.0,78515.0,8546.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8206,2021-03-20T17:00:00,19,Sicilia,731,122,853,14880,15733,-51,782,...,165140,2925797,1228369.0,6.0,165140.0,0.0,1911228.0,1014569.0,ITG,ITG1
8207,2021-03-20T17:00:00,9,Toscana,1424,240,1664,24387,26051,412,1510,...,180998,3178301,1667979.0,9.0,179526.0,1472.0,2754816.0,423485.0,ITI,ITI1
8208,2021-03-20T17:00:00,10,Umbria,372,74,446,5133,5579,-102,206,...,49082,925972,330080.0,4.0,49082.0,0.0,757297.0,168675.0,ITI,ITI2
8209,2021-03-20T17:00:00,2,Valle d'Aosta,25,2,27,476,503,50,61,...,8575,87061,48137.0,0.0,8529.0,46.0,78515.0,8546.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-03-20', '2021-03-19')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-20T17:00:00,1,Piemonte,3361,332,3693,30304,33997,542,2141,...,287164,3058664,1427645.0,23.0,275852.0,11312.0,2274627.0,784037.0,ITC,ITC1
1,2021-03-20T17:00:00,2,Valle d'Aosta,25,2,27,476,503,50,61,...,8575,87061,48137.0,0.0,8529.0,46.0,78515.0,8546.0,ITC,ITC2
2,2021-03-20T17:00:00,3,Lombardia,6891,791,7682,96769,104451,3547,4810,...,693917,7647654,3391168.0,54.0,669097.0,24820.0,6935892.0,711762.0,ITC,ITC4
3,2021-03-20T17:00:00,5,Veneto,1372,205,1577,36756,38333,720,2044,...,364969,5794287,1497373.0,22.0,357092.0,7877.0,4352538.0,1441749.0,ITH,ITH3
4,2021-03-20T17:00:00,6,Friuli Venezia Giulia,564,80,644,15540,16184,321,906,...,91077,1530951,548238.0,6.0,80895.0,10182.0,1373890.0,157061.0,ITH,ITH4
5,2021-03-20T17:00:00,7,Liguria,585,65,650,5266,5916,123,458,...,84759,1165664,490923.0,5.0,84759.0,0.0,1009848.0,155816.0,ITC,ITC3
6,2021-03-20T17:00:00,8,Emilia-Romagna,3519,401,3920,69519,73439,652,2560,...,314526,4547328,1633925.0,34.0,314327.0,199.0,3812106.0,735222.0,ITH,ITH5
7,2021-03-20T17:00:00,9,Toscana,1424,240,1664,24387,26051,412,1510,...,180998,3178301,1667979.0,9.0,179526.0,1472.0,2754816.0,423485.0,ITI,ITI1
8,2021-03-20T17:00:00,10,Umbria,372,74,446,5133,5579,-102,206,...,49082,925972,330080.0,4.0,49082.0,0.0,757297.0,168675.0,ITI,ITI2
9,2021-03-20T17:00:00,11,Marche,755,140,895,9131,10026,-9,856,...,82473,953305,581213.0,8.0,82473.0,0.0,862260.0,91045.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-19T17:00:00,1,Piemonte,3332,326,3658,29797,33455,1295,2997,...,285023,3027647,1418191.0,24.0,274109.0,10914.0,2264029.0,763618.0,ITC,ITC1
1,2021-03-19T17:00:00,2,Valle d'Aosta,24,2,26,427,453,55,63,...,8514,86323,47805.0,0.0,8480.0,34.0,78313.0,8010.0,ITC,ITC2
2,2021-03-19T17:00:00,3,Lombardia,6796,788,7584,93320,100904,1835,5518,...,689107,7591271,3376026.0,52.0,666207.0,22900.0,6897026.0,694245.0,ITC,ITC4
3,2021-03-19T17:00:00,5,Veneto,1353,199,1552,36061,37613,675,1917,...,362925,5748890,1489217.0,24.0,355308.0,7617.0,4332655.0,1416235.0,ITH,ITH3
4,2021-03-19T17:00:00,6,Friuli Venezia Giulia,547,77,624,15239,15863,337,910,...,90171,1519614,543981.0,7.0,80275.0,9896.0,1366491.0,153123.0,ITH,ITH4
5,2021-03-19T17:00:00,7,Liguria,578,63,641,5152,5793,-219,427,...,84301,1158435,488169.0,5.0,84301.0,0.0,1005367.0,153068.0,ITC,ITC3
6,2021-03-19T17:00:00,8,Emilia-Romagna,3549,386,3935,68852,72787,2349,3188,...,311988,4510779,1627949.0,27.0,311793.0,195.0,3790375.0,720404.0,ITH,ITH5
7,2021-03-19T17:00:00,9,Toscana,1448,246,1694,23945,25639,301,1365,...,179488,3153257,1656862.0,16.0,178094.0,1394.0,2739233.0,414024.0,ITI,ITI1
8,2021-03-19T17:00:00,10,Umbria,385,75,460,5221,5681,-130,182,...,48876,920579,329363.0,5.0,48876.0,0.0,754505.0,166074.0,ITI,ITI2
9,2021-03-19T17:00:00,11,Marche,749,141,890,9145,10035,-108,822,...,81617,946689,577354.0,11.0,81617.0,0.0,856765.0,89924.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-03-20T17:00:00,1,Piemonte,3361,332,3693,30304,33997,542,2141,...,3058664,1427645.0,23.0,275852.0,11312.0,2274627.0,784037.0,ITC,ITC1,4.341375e+06
1,2021-03-20T17:00:00,2,Valle d'Aosta,25,2,27,476,503,50,61,...,87061,48137.0,0.0,8529.0,46.0,78515.0,8546.0,ITC,ITC2,1.255010e+05
2,2021-03-20T17:00:00,3,Lombardia,6891,791,7682,96769,104451,3547,4810,...,7647654,3391168.0,54.0,669097.0,24820.0,6935892.0,711762.0,ITC,ITC4,1.010397e+07
3,2021-03-20T17:00:00,5,Veneto,1372,205,1577,36756,38333,720,2044,...,5794287,1497373.0,22.0,357092.0,7877.0,4352538.0,1441749.0,ITH,ITH3,4.907704e+06
4,2021-03-20T17:00:00,6,Friuli Venezia Giulia,564,80,644,15540,16184,321,906,...,1530951,548238.0,6.0,80895.0,10182.0,1373890.0,157061.0,ITH,ITH4,1.211357e+06
5,2021-03-20T17:00:00,7,Liguria,585,65,650,5266,5916,123,458,...,1165664,490923.0,5.0,84759.0,0.0,1009848.0,155816.0,ITC,ITC3,1.543127e+06
6,2021-03-20T17:00:00,8,Emilia-Romagna,3519,401,3920,69519,73439,652,2560,...,4547328,1633925.0,34.0,314327.0,199.0,3812106.0,735222.0,ITH,ITH5,4.467118e+06
7,2021-03-20T17:00:00,9,Toscana,1424,240,1664,24387,26051,412,1510,...,3178301,1667979.0,9.0,179526.0,1472.0,2754816.0,423485.0,ITI,ITI1,3.722729e+06
8,2021-03-20T17:00:00,10,Umbria,372,74,446,5133,5579,-102,206,...,925972,330080.0,4.0,49082.0,0.0,757297.0,168675.0,ITI,ITI2,8.802850e+05
9,2021-03-20T17:00:00,11,Marche,755,140,895,9131,10026,-9,856,...,953305,581213.0,8.0,82473.0,0.0,862260.0,91045.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-03-20T17:00:00,1,Piemonte,3361,332,3693,30304,33997,542,2141,...,784037.0,ITC,ITC1,4.341375e+06,Piemonte,3361,5824,332,628,99
1,2021-03-20T17:00:00,2,Valle d'Aosta,25,2,27,476,503,50,61,...,8546.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,25,227,2,20,12
2,2021-03-20T17:00:00,3,Lombardia,6891,791,7682,96769,104451,3547,4810,...,711762.0,ITC,ITC4,1.010397e+07,Lombardia,6891,13387,791,1416,114
3,2021-03-20T17:00:00,5,Veneto,1372,205,1577,36756,38333,720,2044,...,1441749.0,ITH,ITH3,4.907704e+06,Veneto,1372,6000,205,1000,0
4,2021-03-20T17:00:00,6,Friuli Venezia Giulia,564,80,644,15540,16184,321,906,...,157061.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,564,1277,80,175,0
5,2021-03-20T17:00:00,7,Liguria,585,65,650,5266,5916,123,458,...,155816.0,ITC,ITC3,1.543127e+06,Liguria,585,1789,65,222,12
6,2021-03-20T17:00:00,8,Emilia-Romagna,3519,401,3920,69519,73439,652,2560,...,735222.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,3519,6490,401,760,0
7,2021-03-20T17:00:00,9,Toscana,1424,240,1664,24387,26051,412,1510,...,423485.0,ITI,ITI1,3.722729e+06,Toscana,1424,5033,240,599,20
8,2021-03-20T17:00:00,10,Umbria,372,74,446,5133,5579,-102,206,...,168675.0,ITI,ITI2,8.802850e+05,Umbria,372,860,74,139,2
9,2021-03-20T17:00:00,11,Marche,755,140,895,9131,10026,-9,856,...,91045.0,ITI,ITI3,1.518400e+06,Marche,755,1176,140,243,30


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-03-20T17:00:00,3,Lombardia,6891,791,7682,96769,104451,3547,4810,...,55.86,3449,-708,56383,15142.0,78,1185,8.53,0.04761,6.87
1,2021-03-20T17:00:00,8,Emilia-Romagna,3519,401,3920,69519,73439,652,2560,...,52.76,667,-628,36549,5976.0,47,1839,7.00,0.05731,7.04
2,2021-03-20T17:00:00,15,Campania,1575,161,1736,99159,100895,557,2196,...,25.97,545,199,22090,13521.0,28,1611,9.94,0.03795,5.48
3,2021-03-20T17:00:00,1,Piemonte,3361,332,3693,30304,33997,542,2141,...,52.87,507,-856,31017,9454.0,52,1547,6.90,0.04932,6.61
4,2021-03-20T17:00:00,5,Veneto,1372,205,1577,36756,38333,720,2044,...,20.50,695,127,45397,8156.0,20,1304,4.50,0.04165,7.44
5,2021-03-20T17:00:00,16,Puglia,1668,209,1877,40195,42072,902,1983,...,36.73,869,198,11296,5415.0,26,1055,17.55,0.04947,4.37
6,2021-03-20T17:00:00,12,Lazio,2484,309,2793,44261,47054,1059,1821,...,32.77,1013,-367,37419,21163.0,24,738,4.87,0.03105,4.54
7,2021-03-20T17:00:00,9,Toscana,1424,240,1664,24387,26051,412,1510,...,40.07,442,145,25044,11117.0,18,1080,6.03,0.04056,4.86
8,2021-03-20T17:00:00,6,Friuli Venezia Giulia,564,80,644,15540,16184,321,906,...,45.71,301,-4,11337,4257.0,14,571,7.99,0.07479,7.52
9,2021-03-20T17:00:00,11,Marche,755,140,895,9131,10026,-9,856,...,57.61,-14,34,6616,3859.0,20,845,12.94,0.05638,5.43


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,52,38,90,57.78
1,Marche,140,103,243,57.61
2,Lombardia,791,625,1416,55.86
3,Umbria,74,65,139,53.24
4,Piemonte,332,296,628,52.87
5,Emilia-Romagna,401,359,760,52.76
6,Friuli Venezia Giulia,80,95,175,45.71
7,Molise,16,23,39,41.03
8,Toscana,240,359,599,40.07
9,Abruzzo,83,132,215,38.60


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))